<a href="https://colab.research.google.com/github/Noufal21/text-mining/blob/master/project/Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile
file_name = "/content/filter_train_stem_ASCII.csv.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("done")

done


In [ ]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-12-17 19:35:55--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.86.126
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.86.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  16.4MB/s    in 98s     

2020-12-17 19:37:34 (16.1 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
!wget -c "http://nlp.stanford.edu/data/glove.840B.300d.zip"

--2020-12-17 21:50:57--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2020-12-17 21:50:57--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2020-12-17 21:50:58--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

In [ ]:
from zipfile import ZipFile
file_name = "/content/glove.840B.300d.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("done")

done


In [ ]:
!pip install fuzzywuzzy
!pip install distance
import warnings
warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  after removing the cwd from sys.path.


In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.attrs import ORTH, LEMMA, NORM, TAG
from spacy.language import Language
from spacy.tokens import Doc
from spacy.symbols import IS_CURRENCY
from fuzzywuzzy import fuzz
import distance
import gensim
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from scipy.stats import skew, kurtosis
from sklearn.metrics import pairwise_distances
import codecs
from gensim.models.fasttext import FastText

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])

In [ ]:
def tokenizeWord(question1,question2):
  q1_token = nlp(question1)
  q2_token = nlp(question2)



  q1_token_set = set([word.lemma_ for word in q1_token if not word.is_punct and not word.is_space])
  q2_token_set = set([word.lemma_ for word in q2_token if not word.is_punct and not word.is_space])
  
  q1_words = set([word.lemma_ for word in q1_token  if not word.is_space and not word.is_punct and not word.is_stop])
  q1_stop = set([word.lemma_ for word in q1_token  if not word.is_space and not word.is_punct and  word.is_stop])
  
  q2_words = set([word.lemma_ for word in q2_token  if not word.is_space and not word.is_punct and not word.is_stop])
  q2_stop = set([word.lemma_ for word in q2_token  if not word.is_space and not word.is_punct and  word.is_stop])
  
  common_word_count = len(q1_words.intersection(q2_words))
  common_stop_count = len(q1_stop.intersection(q2_stop))
  common_token_count = len(q1_token_set.intersection(q2_token_set))


  question1 = " ".join([word.text for word in q1_token if not word.is_punct and not word.is_space])
  question2 = " ".join([word.text for word in q2_token if not word.is_punct and not word.is_space])

  q1_token = nlp(question1)
  q2_token = nlp(question2)
  return q1_token, q2_token, q1_words, q1_stop, q2_words, q2_stop, common_word_count, common_stop_count, common_token_count

def get_longest_substr_ratio(q1, q2):
    strs = list(distance.lcsubstrings(q1, q2))
    if len(strs) == 0:
        return 0
    else:
        return len(strs[0])

def sent2vec(words,model):
    M = []
    for w in words:
      try:
        M.append(model[w])
      except:
          continue
    M = np.array(M)
    v = M.sum(axis=0)
    norm = np.sqrt((v ** 2).sum())
    if norm > 0:
      return v/norm
    else:
      return None

def import_embeddingModel():
  pass

def feature_extraction(question1, question2, word2vecModel, word2vecModel_norm, gloveModel,glove_norm):

  q1_token, q2_token, q1_words, q1_stop, q2_words, q2_stop, common_word_count, common_stop_count, common_token_count = tokenizeWord(question1.lower(), question2.lower())
  if len(q1_token) == 0 or len(q2_token)  == 0:
    return [0] * 31
  cwc_min = common_word_count / (min(len(q1_words),len(q2_words))+ 0.0001)
  cwc_max = common_word_count / (max(len(q1_words),len(q2_words))+ 0.0001)

  csc_min = common_stop_count / (min(len(q1_stop) ,len(q2_stop)) + 0.0001)
  csc_max = common_stop_count / (max(len(q1_stop),len(q2_stop))+ 0.0001)

  ctc_min = common_token_count / (min(len(q1_token) ,len(q2_token))+ 0.0001)
  ctc_max = common_token_count / (max(len(q1_token),len(q2_token))+ 0.0001)

  last_word_eq = int(q1_token[-1].text == q2_token[-1].text)
  first_word_eq = int(q1_token[0].text == q2_token[0].text)
  abs_len_diff  = abs(len(q1_token) - len(q2_token))
  mean_len= (len(q1_token) + len(q2_token))/2

  # Fuzz features

  fuzz_qratio =  fuzz.QRatio(question1, question2)
  fuzz_WRatio = fuzz.WRatio(question1, question2)
  fuzz_partial_ratio = fuzz.partial_ratio(question1, question2)
  fuzz_partial_token_set_ratio = fuzz.partial_token_set_ratio(question1,question2)
  fuzz_partial_token_sort_ratio = fuzz.partial_token_sort_ratio(question1, question2)
  fuzz_token_set_ratio = fuzz.token_set_ratio(question1, question2)
  fuzz_token_sort_ratio = fuzz.token_sort_ratio(question1, question2)

  longest_substr_ratio = get_longest_substr_ratio(question1,question2)/ (min(len(question1), len(question2))+ 0.0001)

  q1_token = nlp(question1)
  q2_token = nlp(question2)

  question1 = [word.text for word in q1_token if not word.is_punct and not word.is_space and not word.is_stop]
  question2 = [word.text for word in q2_token if not word.is_punct and not word.is_space and not word.is_stop]


  # Embedding Features
  if gloveModel == None:
    wmd = min(word2vecModel.wmdistance(question1, question2), 10)
    wmd_norm = min(word2vecModel_norm.wmdistance(question1, question2), 10)


    q1vec = sent2vec(question1,word2vecModel)
    q2vec = sent2vec(question2,word2vecModel)

    if q1vec is not None and q2vec is not None:
      cos = cosine(q1vec, q2vec)
      city = cityblock(q1vec, q2vec)
      jacc = jaccard(q1vec, q2vec)
      canb = canberra(q1vec, q2vec)
      eucl = euclidean(q1vec, q2vec)
      mink = minkowski(q1vec, q2vec, 3)
      bray = braycurtis(q1vec, q2vec)

      q1_skew = skew(q1vec)
      q2_skew = skew(q2vec)
      q1_kurt = kurtosis(q1vec)
      q2_kurt = kurtosis(q2vec)

    else:
      cos = city = jacc = canb = eucl = mink = bray = -1
      q1_skew = q2_skew = q1_kurt = q2_kurt = 0


  else: 
    wmd = min(gloveModel.wmdistance(question1, question2), 10)
    wmd_norm = min(glove_norm.wmdistance(question1, question2), 10)
    q1vec = sent2vec(question1,gloveModel)
    q2vec = sent2vec(question2,gloveModel)
    
    if q1vec is not None and q2vec is not None:
      cos = cosine(q1vec, q2vec)
      city = cityblock(q1vec, q2vec)
      jacc = jaccard(q1vec, q2vec)
      canb = canberra(q1vec, q2vec)
      eucl = euclidean(q1vec, q2vec)
      mink = minkowski(q1vec, q2vec, 3)
      bray = braycurtis(q1vec, q2vec)

      q1_skew = skew(q1vec)
      q2_skew = skew(q2vec)
      q1_kurt = kurtosis(q1vec)
      q2_kurt = kurtosis(q2vec)

    else:
      cos = city = jacc = canb = eucl = mink = bray = -1
      q1_skew = q2_skew = q1_kurt = q2_kurt = 0

  #print(cwc_min)
  return [cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,
          mean_len,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,
          fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio,
          longest_substr_ratio,wmd,wmd_norm,cos,city,jacc,canb,eucl,mink,bray,q1_skew,
          q2_skew,q1_kurt,q2_kurt]




  

In [ ]:
embedding_file = "/content/GoogleNews-vectors-negative300.bin.gz"
w2v = gensim.models.KeyedVectors.load_word2vec_format(embedding_file, binary=True)
w2v_norm = gensim.models.KeyedVectors.load_word2vec_format(embedding_file, binary=True)
w2v_norm.init_sims(replace=True)

In [ ]:
train_df = pd.read_csv("/content/filter_train_stem_ASCII.csv",index_col = False)
train_df = train_df.dropna()
train_df = train_df.sort_values('id').reset_index()
train_df.head(12)

,index,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate
0,0,0,0,1,2,what is the step by step guid to invest in sha...,what is the step by step guid to invest in sha...,0
1,1,1,1,3,4,what is the stori of kohinoor koh i noor diamond,what would happen if the indian govern stole t...,0
2,2,2,2,5,6,how can i increas the speed of my internet con...,how can internet speed be increas by hack thro...,0
3,3,3,3,7,8,whi am i mental veri lonely how can i solv it,find the remaind when math]23^{24}[/math is di...,0
4,4,4,4,9,10,which one dissolv in water quik sugar salt met...,which fish would surviv in salt water,0
5,5,5,5,11,12,astrology i am a capricorn sun cap moon and ca...,i am a tripl capricorn sun moon and ascend in ...,1
6,6,6,6,13,14,should i buy tiago,what keep childern activ and far from phone an...,0
7,7,7,7,15,16,how can i be a good geologist,what should i do to be a great geologist,1
8,8,8,8,17,18,when do you use instead of,when do you use instead of and,0
9,9,9,9,19,20,motorola company can i hack my charter motorol...,how do i hack motorola dcx3400 for free internet,0


In [ ]:
t = [feature_extraction(q1, q2, None, None, glove,glove_norm) for q1,q2 in train_df[["question1","question2"]].to_numpy()]


In [ ]:
columns = ["cwc_min","cwc_max","csc_min","csc_max","ctc_min","ctc_max","last_word_eq","first_word_eq","abs_len_diff",
          "mean_len","fuzz_qratio","fuzz_WRatio","fuzz_partial_ratio",'fuzz_partial_token_set_ratio',
          "fuzz_partial_token_sort_ratio","fuzz_token_set_ratio","fuzz_token_sort_ratio",
          "longest_substr_ratio","wmd","wmd_norm","cos","city","jacc","canb",'eucl','mink',"bray","q1_skew",
          "q2_skew",'q1_kurt',"q2_kurt"]

token_features = pd.DataFrame(t,columns=columns)
token_features.head(12)

,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio,longest_substr_ratio,wmd,wmd_norm,cos,city,jacc,canb,eucl,mink,bray,q1_skew,q2_skew,q1_kurt,q2_kurt
0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0,1,2,13.0,92,95,100,100,87,100,92,0.999998,1.242895,0.182337,0.037739,3.757679,1.0,106.020161,0.274731,0.125297,0.176137,1.018397,1.196996,18.915699,18.310662
1,0.799984,0.499994,0.599988,0.499992,0.699993,0.466664,0,1,5,12.5,67,86,71,100,73,86,65,0.583332,4.568035,0.677708,0.219182,7.389543,1.0,125.862633,0.662090,0.415619,0.307388,-0.181118,-1.012365,1.653170,7.752329
2,0.599988,0.499992,0.399992,0.249997,0.499995,0.357140,0,1,4,12.0,39,68,38,100,70,72,64,0.188679,3.500375,0.536211,0.131042,6.912877,1.0,129.852605,0.511940,0.239491,0.296184,-0.027762,-0.532948,11.047251,5.152646
3,0.000000,0.000000,0.249994,0.199996,0.111110,0.090908,0,0,2,10.0,23,28,26,35,32,30,26,0.044444,7.531046,1.188850,0.592328,15.554620,1.0,199.626794,1.088419,0.482211,0.722758,-0.683867,-0.727365,2.910043,2.424369
4,0.499988,0.222220,0.666644,0.499988,0.571420,0.307690,0,1,6,10.0,36,86,54,100,71,68,49,0.162162,6.734644,0.907757,0.320806,10.878496,1.0,164.558107,0.801007,0.364791,0.480040,-0.130439,-0.079190,6.468015,3.385341
5,0.666656,0.571420,0.888879,0.799992,0.705878,0.705878,1,0,0,17.0,75,79,74,100,77,83,77,0.185185,4.072657,0.609093,0.204045,8.393666,1.0,144.849014,0.638819,0.314174,0.365167,-0.831551,-0.893273,9.887077,5.285349
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,7,7.5,18,35,39,39,39,25,23,0.111110,8.246431,1.274807,0.751018,16.119991,1.0,201.499913,1.225576,0.584168,0.812378,-0.153566,-0.930680,0.347224,13.224931
7,0.499975,0.499975,0.599988,0.428565,0.571420,0.444440,1,0,2,8.0,61,67,66,100,69,71,61,0.344826,1.510052,0.281362,0.054163,4.542877,1.0,100.792654,0.329130,0.150444,0.183996,1.048181,1.105783,8.640427,8.392096
8,0.999950,0.999950,0.999975,0.799984,0.999983,0.857131,0,1,1,6.5,93,95,100,100,100,100,93,0.999996,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.171160,1.171160,16.944106,16.944106
9,0.599988,0.499992,0.333322,0.249994,0.444440,0.444440,0,0,0,9.0,50,62,56,100,55,65,44,0.208333,4.897002,0.721900,0.241410,9.754137,1.0,153.969825,0.694852,0.316839,0.407277,-0.367810,-0.379719,2.199797,7.115269


In [ ]:
token_features.to_pickle("/content/stem_train_ASCII_features_glove")

In [ ]:
# -*- coding: utf-8 -*-
"""
word2vec embeddings start with a line with the number of lines (tokens?) and 
the number of dimensions of the file. This allows gensim to allocate memory 
accordingly for querying the model. Larger dimensions mean larger memory is 
held captive. Accordingly, this line has to be inserted into the GloVe 
embeddings file.
"""

import os
import shutil
import smart_open
from sys import platform

import gensim


def prepend_line(infile, outfile, line):
	""" 
	Function use to prepend lines using bash utilities in Linux. 
	(source: http://stackoverflow.com/a/10850588/610569)
	"""
	with open(infile, 'r') as old:
		with open(outfile, 'w') as new:
			new.write(str(line) + "\n")
			shutil.copyfileobj(old, new)

def prepend_slow(infile, outfile, line):
	"""
	Slower way to prepend the line by re-creating the inputfile.
	"""
	with open(infile, 'r') as fin:
		with open(outfile, 'w') as fout:
			fout.write(line + "\n")
			for line in fin:
				fout.write(line)

def get_lines(glove_file_name):
    """Return the number of vectors and dimensions in a file in GloVe format."""
    with smart_open.smart_open(glove_file_name, 'r') as f:
        num_lines = sum(1 for line in f)
    with smart_open.smart_open(glove_file_name, 'r') as f:
        num_dims = len(f.readline().split()) - 1
    return num_lines, num_dims
	
# Input: GloVe Model File
# More models can be downloaded from http://nlp.stanford.edu/projects/glove/
glove_file="/content/glove.840B.300d.txt"

num_lines, dims = get_lines(glove_file)

# Output: Gensim Model text format.
gensim_file='glove_model2.txt'
gensim_first_line = "{} {}".format(num_lines, dims)

# Prepends the line.
if platform == "linux" or platform == "linux2":
	prepend_line(glove_file, gensim_file, gensim_first_line)
else:
	prepend_slow(glove_file, gensim_file, gensim_first_line)

# Demo: Loads the newly created glove_model.txt into gensim API.


In [ ]:
glove = gensim.models.KeyedVectors.load_word2vec_format(gensim_file,binary=False) #GloVe Model
glove_norm = gensim.models.KeyedVectors.load_word2vec_format(gensim_file, binary=False)
glove_norm.init_sims(replace=True)

In [ ]:
glove.similarity('woman', 'man')